### Import Modules and Data

In [1]:
import pandas as pd
import plotly.express as px

pd.options.display.max_colwidth = 1000
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', None)

In [245]:
data = pd.read_csv('data/CHG-CEE-20_clean.csv')
ab_lit = pd.read_csv('data/aboriginal_literacy.csv')

### Charts

#### Gender

In [94]:
gender = data['gender'].value_counts()
gender = gender.reset_index(name='count').rename(columns={"index": "gender"})
gender['percentage'] = gender['count']/gender['count'].sum()
gender['id'] = 1

gender

,gender,count,percentage,id
0,Female,37,0.74,1
1,Male,12,0.24,1
2,Other,1,0.02,1


In [146]:
colors = ['#F8B500', '#F47721', '#82C441', '#0090B4', '#595959']

customdata  = np.stack((gender['gender'], gender['count']), axis=-1)

hovertemplate = ("<b>%{customdata[0]}</b><br>" +
                  "Percentage: %{x:.0%}<br>" +
                  "Count: %{customdata[1]}<br>" +
                  "<extra></extra>")


fig = px.bar(gender,
             x = 'percentage',
             y='id',
             color='gender',
             barmode = 'stack',
             orientation='h',
             template="simple_white",
             text = "<b>" + gender['gender'] + "</b>" + "<br>" +
                    gender['percentage'].apply(lambda x: '{0:1.0f}%'.format(x * 100)) +
                    "<br>" + '(' + gender['count'].astype(str) + ')',
             hover_data={'id':False, 'gender': False, 'count':True},
             height = 300,
             color_discrete_sequence=colors)

fig.update_traces(textposition='inside',
                  insidetextanchor= 'middle',
                  textfont=dict(family="san serif",
                                color="white"),
                  hovertemplate=hovertemplate)
                  
fig.update_yaxes(visible=False)

fig.update_xaxes(title = '',
                 tickformat = ",.0%",
                 showticklabels= True,)

fig.update_layout(uniformtext_minsize=15,
                  uniformtext_mode='hide',
                  legend= dict(title= None,
                               bgcolor='#F6F5F4',
                               orientation = 'h',
                               yanchor="bottom",
                               y=-.5,
                               xanchor="center",
                               x=.5))
fig.show()

#### Aboriginal Literacy

In [246]:
ab_lit = ab_lit[['Question', 'Response']]

In [270]:
ab_lit_counts = ab_lit.value_counts().to_frame()
ab_lit_counts.reset_index(inplace=True)
ab_lit_counts.rename(columns={0:'Count'}, inplace=True)

In [271]:
grouped_df = ab_lit_counts.groupby(['Question', 'Response']).agg({'Count': 'sum'})
# Change: groupby state_office and divide by sum
# question_pcts = grouped_df.groupby(level=0).apply(lambda x: x / float(x.sum()))

grouped_df['Percentage'] = grouped_df['Count'] / grouped_df.groupby('Question')['Count'].transform('sum')
grouped_df.reset_index(drop=False, inplace=True)

In [288]:
colors = ['#F8B500', '#F47721', '#82C441', '#0090B4', '#595959']

customdata  = np.stack((grouped_df['Response'], grouped_df['Count']), axis=-1)

hovertemplate = ("<b>%{customdata[0]}</b><br>" +
                  "Percentage: %{x:.0%}<br>" +
                  "Count: %{customdata[1]}<br>" +
                  "<extra></extra>")


fig = px.bar(grouped_df,
             x = 'Percentage',
             y='Question',
             color='Response',
             barmode = 'stack',
             orientation='h',
             template="simple_white",
             text = "<b>" + grouped_df['Response'] + "</b>" + "<br>" +
                    grouped_df['Percentage'].apply(lambda x: '{0:1.0f}%'.format(x * 100)) +
                    "<br>" + '(' + grouped_df['Count'].astype(str) + ')',
             hover_data={'Question':True, 'Response': True, 'Percentage': True, 'Count':True},
             height = 700,
             color_discrete_sequence=colors)

fig.update_traces(textposition='inside',
                  insidetextanchor= 'middle',
                  textfont=dict(family="san serif",
                                color="white"),
                  hovertemplate=hovertemplate)
                  
fig.update_yaxes(title = '',
                 showticklabels= True,)
fig.update_xaxes(title = '',
                 tickformat = ",.0%",
                 showticklabels= True,)

fig.update_layout(uniformtext_minsize=15,
                  uniformtext_mode='hide',
                  legend= dict(title= None,
                               bgcolor='#F6F5F4',
                               orientation = 'h',
                               yanchor="top",
                               y=1.05,
                               xanchor="center",
                               x=.5))
fig.show()